In [ ]:
# URL Shortener Project - Full Code

!pip install flask pyngrok

from flask import Flask, request, redirect, render_template_string
from pyngrok import ngrok
import sqlite3, string, random

ngrok.set_auth_token("35WepmSHt4R6D65b6kC5HcXZy76_3DX17a6axgFHCxBNVuLeQ")

conn = sqlite3.connect('url_shortener.db', check_same_thread=False)
cursor = conn.cursor()

cursor.execute('''
CREATE TABLE IF NOT EXISTS urls (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    short_code TEXT UNIQUE,
    original_url TEXT,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    access_count INTEGER DEFAULT 0
)
''')
conn.commit()

def generate_short_code(length=6):
    return ''.join(random.choices(string.ascii_letters + string.digits, k=length))

app = Flask(__name__)

HTML_TEMPLATE = """<!DOCTYPE html>
<html>
<head><title>URL Shortener</title></head>
<body>
<form method='POST'>
<input name='url' placeholder='Enter URL'>
<button type='submit'>Shorten</button>
</form>
{% if short_url %}
<p>Short URL: <a href='{{ short_url }}'>{{ short_url }}</a></p>
{% endif %}
</body></html>"""

@app.route('/', methods=['GET','POST'])
def home():
    short_url=None
    if request.method=='POST':
        long_url=request.form['url']
        code=generate_short_code()
        cursor.execute("INSERT INTO urls (short_code, original_url) VALUES (?,?)",(code,long_url))
        conn.commit()
        short_url=public_url + "/" + code
    return render_template_string(HTML_TEMPLATE, short_url=short_url)

@app.route('/<code>')
def go(code):
    cursor.execute("SELECT original_url FROM urls WHERE short_code=?",(code,))
    row=cursor.fetchone()
    if row: return redirect(row[0])
    return "Invalid short URL",404

public_url = ngrok.connect(5000)
print("Live Shortener:", public_url)
app.run(port=5000)
